In [37]:
import pandas as pd
import cobra
import re
import collections
from cobra.io.dict import model_to_dict

In [38]:
def model2excel(mname):#将模型文件转为excel文件，以供之后的程序取用模型信息
    writer = pd.ExcelWriter(mname+'.xlsx')
    pd.DataFrame(a['metabolites']).to_excel(writer,'metabolites',index=False)
    pd.DataFrame(a['genes']).to_excel(writer,'genes',index=False)
    df_r=pd.DataFrame(a['reactions'])
    df_r['reaction_eq'] = df_r.id.apply(lambda x: model.reactions.get_by_id(x).build_reaction_string(use_metabolite_names=False))
    df_r.to_excel(writer,'reactions',index=False)
    writer.save()

In [39]:
def getmeta(model):#从模型文件中提取代谢物的id信息
    each_meta=[]
    for line in model.metabolites:
        with model as model:
            each_meta.append(line.id)
    return each_meta

def uniq(a, b):#功能函数：给一个列表去重
    for i in a:
        if i not in b:
            b.append(i)
    return b

In [40]:
#利用模型文件，依据代谢物的分子式对代谢物进行简化，将所有模型中的代谢物都替换为简化形式，若代谢物没有简化形式，则标记出来
def getCNpattern(each_meta,model):
    meta_list=[]
    uniq_metalist=[]
    for meta in each_meta:
        dc = {}
        dc = collections.OrderedDict()
        changed=meta
        formu =model.metabolites.get_by_id(changed)
        mf=re.findall (r'([A-Z][a-z]*)(\d*)', str(formu.formula))
        cv = []
        nv = []
        for one in mf:
            full_value=[]
            if 'C' in one:
                if one[1] == '':
                    value=1
                else:
                    value=one[1]
                cv='C' + str(value)
            if 'N' in one:
                if one[1] == '':
                    value=1
                else:
                    value=one[1]
                nv= 'N' + str(value)
            full_value =str(cv) + str(nv)
            print(full_value)
        dc['id'] = changed
        if full_value.replace('[]', '')=='':
            dc['CxNy'] ='-'
        else:
            dc['CxNy'] = full_value.replace('[]', '')
            uniq_metalist.append(full_value.replace('[]', ''))
        meta_list.append(dc)
    return meta_list,uniq_metalist

In [41]:
def duiyingdic(data1):#从简化后的代谢物列表中提取代谢物id和简化形式的对应关系
    global dict_id_name
    dict_id_name = {}
    for line in data1:
        if line['id'] not in dict_id_name:
            dict_id_name[line['id']] = line['CxNy']


In [42]:
def fine_cum():#根据提供的流通代谢物的列表，从中提取流通代谢物的id信息，在替换反应方程时会用到
    global dict_cum_name
    dict_cum_name = []
    excel1 = pd.ExcelFile('流通代谢物整合.xlsx')
    data1 = excel1.parse('Sheet2', index_col=None).T.to_dict().values()
    for line in data1:
        dict_cum_name.append(line['大肠id'])
    #print(dict_cum_name)


In [43]:
# 并根据提取出的代谢物id和简化形式的对应关系，将代谢物替换后再返回，以完成反应方程的简化形式的替换
def reaP(left_or_right, momets, metsnopar):
    if left_or_right == '':
        return left_or_right
    prelist = left_or_right.split(' + ')
    metlist = []
    for item in prelist:
        tool = item
        tool = tool.replace('_c', '').replace('_e', '').replace('_p', '')
        if tool not in dict_cum_name:
            metlist.append(item)
    for n, i in enumerate(metlist):
        if ' ' not in i:
            try:
                to_i = dict_id_name[i]  # search and replace
                to_i = str(to_i)
                if to_i == '' or to_i == ' ':
                    to_i = i
                    metsnopar.add(i)
            except:
                to_i = i
                momets.add(i)
            i = i.replace(i, to_i)
        else:
            i2 = re.sub('.*? ', '', i)  # 替换函数
            try:
                to_i = dict_id_name[i2]  ##search and replace
                if to_i == '' or to_i == ' ':
                    to_i = i2
                    metsnopar.add(i2)
            except:
                to_i = i2
                momets.add(i2)
            i = i.replace(i2, to_i)

        if i == '':
            metlist[n] = i
        else:
            metlist[n] = i
    list2 = []
    for item in metlist:
        if '-' not in item:
            list2.append(item)
    return (' + '.join(list2), list2)

In [44]:
#将反应方程式进行拆分，拆分为代谢物，之后利用‘reaP’函数将这些代谢物替换为对应的简化形式，返回替换后的反应方程
#计算结果分析——替换名字
def replace_eq(reaction_eq):
    tm=reaction_eq
    duiying=r'D:/different_database/metacyc/metacyc_slim/result file/'
    fine_cum()
    momets = set()
    metsnopar = set()
    splitstr=[]
    if ' --> ' in tm:
        splitstr = ' --> '
    elif ' <-- ' in tm:
        splitstr = ' <-- '
    elif ' <=> ' in tm:
        splitstr = ' <=> '
    elif ' ->' in tm:
        splitstr = ' ->'
    else:
        print('需要修正反应方程式的连接符')
    mets = tm.split(splitstr)
    try:
        left, leftlist= reaP(mets[0], momets, metsnopar)
        right, rightlist = reaP(mets[1], momets, metsnopar)
    except:
        print("反应替换出错")
    if rightlist == '':
        difference1=leftlist
        difference2=rightlist
    else:
        difference1=[item for item in leftlist if not item in rightlist]
        difference2=[item for item in rightlist if not item in leftlist]
    if difference1 == []:
        rea='()'
        #将可逆符号左右简化形式相同的代谢物相抵消，输出‘（）’以表示
    else:
        rea = ' + '.join(difference1) + splitstr + ' + '.join(difference2)
    tm= rea
    return(str(tm))


In [45]:
if __name__ == '__main__':
#     path = 'D:/different_database/metacyc/metacyc_slim/'  # 输入：模型文件的路径
    mname = 'metacyc18620new'  # 输入：模型文件的名称
    model = cobra.io.read_sbml_model(mname + '.xml')
    a = model_to_dict(model, sort=False)
    # 读取模型文件
    # 提取模型文件中的全部信息
    model2excel(mname)  # 将sbml模型文件转为excel文件
    each_meta = getmeta(model)  # 从模型文件中提取代谢物的id信息
    meta_list, uniq_metalist = getCNpattern(each_meta, model)
    #print(uniq_metalist)

    #writer = pd.ExcelWriter('D:/different_database/metacyc/metacyc_slim/result file/' + '（第一步）简化模型——meta_list.xlsx')
    #pd.DataFrame(meta_list).to_excel(writer, 'meta_CxNy', index=False)
    #writer.save()
    # 利用模型文件，依据代谢物的分子式对代谢物进行简化，先将模型中的所有代谢物的尾标去掉，然后替换为代谢物的简化形式，
    # 若代谢物没有简化形式，则标记出来。将结果分为带有无简化形式的代谢物和不带有两份列表，方便之后工作
    after_uniq = []
    after_uniq = uniq(uniq_metalist, after_uniq)  # 上一步中得到的只含有有简化形式代谢物的列表，对此列表进行去重
    duiyingdic(meta_list)  # 从简化后的代谢物列表中提取代谢物id和简化形式的对应关系
    excel1 = pd.ExcelFile(mname + '.xlsx')
    data1 = excel1.parse('reactions', index_col=None).T.to_dict().values()  # 将dataframe转化为list of dictionaries
    # 读取上述的‘model2excel’函数生成的模型的excel表格文件中的反应列表data1
    print_list = []
    # 从data1中提取出反应id和方程式的信息，利用‘replace_eq’的函数将反应方程进行简化，并将简化后因为代谢物导致的方程为空的反应进行标记。
    for line in data1:
        rdict = {}
        rid = line['id']
        req1 = line['reaction_eq']
        #rid='_2__46__6__46__1__46__14__45__RXN__45__2__45__OXOSUCCINAMATE__47__ASN__47____47__2__45__OXOSUCCINAMATE__47__ASN__46__43__46__'
        #req1='_2__45__OXOSUCCINAMATE_c + ASN_c --> _2__45__OXOSUCCINAMATE_c + ASN_c'
        if 'R_EX_' not in rid and 'EX_Glucose' not in rid:#
            print(rid)
            print(req1)
            req2 = replace_eq(line['reaction_eq'])
            rdict['id'] = rid.replace("['", '').replace("']", '')
            rdict['reaction_eq'] = req1.replace("['", '').replace("']", '')
            rdict['reaction_CNeq'] = req2.replace("['", '').replace("']", '').replace("()", '')
            #print(rdict)
            print_list.append(rdict)
        #break
    #exit()
    print(print_list)
    writer = pd.ExcelWriter('（第一步）简化模型——预处理文件(MC).xlsx')
    pd.DataFrame(meta_list).to_excel(writer, 'meta_CxNy', index=False)
    # 输出代谢物id和代谢物相应简化形式的对应关系（包含所有代谢物）
    pd.DataFrame(after_uniq).to_excel(writer, 'CxNy', header=['CxNy'], index=False)
    # 输出代谢物id和代谢物相应简化形式的对应关系（除去无简化形式的代谢物）
    pd.DataFrame(print_list).to_excel(writer, 'react_CxNy', index=False)
    # 输出反应id、简化后的反应方程和原反应方程的对应关系
    writer.save()
    print('finish!!')

[][]
[][]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C10[]
C10[]
C10N4
C10N4
C10N4
C10[]
C10[]
C10N4
C10N4
C10N4
C10[]
C10[]
C10N4
C10N4
C10[]
C10[]
C10N4
C10N4
C10N4
C5[]
C5[]
C5N4
C5N4
C5[]
C5[]
C5N4
C5N4
[][]
[][]
[][]
[][]
[][]
[][]
[][]
[][]
C21[]
C21[]
C21N2
C21N2
C8[]
C8[]
C8[]
C8[]
C8[]
C8[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5N1
C5N1
C5[]
C5[]
C5N1
C5N1
C9[]
C9[]
C9N2
C9N2
C9N2
C6[]
C6[]
C6N2
C6N2
C9[]
C9[]
C9N2
C9N2
C9[]
C9[]
C9N2
C9N2
C1[]
C1[]
C1N2
C1N2
C5[]
C5[]
C5N4
C5N4
C4[]
C4[]
C4N2
C4N2
C4[]
C4[]
C4N2
C4N2
C9[]
C9[]
C9N2
C9N2
C9N2
C9[]
C9[]
C9N2
C9N2
C9N2
C9[]
C9[]
C9N2
C9N2
C9N2
C17[]
C17[]
C17N3
C17N3
C17N3
C15[]
C15[]
C15N2
C15N2
C15N2
C15[]
C15[]
C15N2
C15N2
C15N2
C14[]
C14[]
C14N2
C14N2
C14N2
C15[]
C15[]
C15N2
C15N2
C15N2
C15[]
C15[]
C15N2
C15N2
C15N2
C54[]
C54[]
C54[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C49[]
C39[]
C39[]
C39[]
C19[]
C19[]
C19[]
C59[]
C59[]
C59[]
C39[]
C39[]
C39[]
C8[]
C8[]
C8N1
C8N1
C9[]
C9[]
C9N1
C9N1
C9[]
C9[]
C9N1


C5[]
C5[]
C5N1
C5N1
C5[]
C5[]
C5N1
C5N1
C5N1
C8[]
C8[]
C8N2
C8N2
C8N2
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C5[]
C5[]
C5N1
C5N1
C9[]
C9[]
C9N1
C9N1
C5[]
C5[]
C5N1
C5N1
C6[]
C6[]
C6[]
C3[]
C3[]
C3N1
C3N1
C3N1
C7[]
C7[]
C7N2
C7N2
C7N2
C6[]
C6[]
C6N3
C6N3
C28[]
C28[]
C28N8
C28N8
C28N8
C28N8
C4[]
C4[]
C4N2
C4N2
C4[]
C4[]
C4N1
C4N1
C4N1
C4[]
C4[]
C4N1
C4N1
C4[]
C4[]
C4N1
C4N1
C6[]
C6[]
C6[]
C6[]
C10[]
C10[]
C10N4
C10N4
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C7[]
C7[]
C7N1
C7N1
C3[]
C3[]
C3N1
C3N1
C3[]
C3[]
C3N1
C3N1
C5[]
C5[]
C5N1
C5N1
C20[]
C20[]
C20[]
C21[]
C21[]
C21[]
C21[]
C44[]
C44[]
C44N1
C44N1
C44N1
C8[]
C8[]
C8[]
C8[]
C8[]
C8[]
C8[]
C27[]
C27[]
C27N7
C27N7
C27N7
C27N7
[][]
[][]
[][]
[][]
C12[]
C12[]
C12[]
C10[]
C10[]
C10N4
C10N4
C10N4
C5[]
C5[]
C5[]
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C14[]
C14[]
C14N3
C14N3
C14N3
C12[]
C12[]
C12[]
C10[]
C10[]
C10[]
C25[]
C25[]
C25N7
C25N7
C25N7
C25N7
C4[]
C4[]
C4[]
C3[]
C3[]
C3[]
[][]
C6[]
C6[]
C6[]
C6[]
C

C52[]
C57[]
C57[]
C57[]
C56[]
C56[]
C56[]
C10[]
C10[]
C10N1
C10N1
C17[]
C17[]
C17N1
C17N1
C12[]
C12[]
C12[]
C55[]
C55[]
C55[]
C40[]
C40[]
C40[]
C45[]
C45[]
C45[]
C51[]
C51[]
C51[]
C51[]
C51[]
C51[]
C51[]
C51[]
C51[]
C41[]
C41[]
C41[]
C66[]
C66[]
C66[]
C71[]
C71[]
C71[]
C76[]
C76[]
C76[]
C56[]
C56[]
C56[]
C61[]
C61[]
C61[]
C46[]
C46[]
C46[]
C44[]
C44[]
C44[]
C55[]
C55[]
C55[]
C55[]
C39[]
C39[]
C39[]
C39[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C24[]
C24[]
C24[]
C7[]
C7[]
C7N1
C7N1
C2[]
C2[]
C2[]
C2[]
C7[]
C7[]
C7[]
C12[]
C12[]
C12[]
C12[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C5[]
C5[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C12[]
C12[]
C12[]
C12[]
C7[]
C7[]
C7[]
C12[]
C12[]
C12[]
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C15[]
C15[]
C15[]
C15[]
C15[]
C15[]
C15[]
C15[]
C15[]
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C26[]
C26[]
C26N7
C26N7
C26N7
C26N7
C6[]
C6[]
C6N1
C6N1
C16[]
C16[]
C16N5
C16N5
C16N5
C7[]
C7[]
C7N2
C7N2
C54[]
C54[]
C54[]
C48[]
C48[]
C48[

C15N5
C15[]
C15[]
C15N5
C15N5
C6[]
C6[]
C6N1
C6N1
C27[]
C27[]
C27[]
C27[]
C27[]
C27[]
C28[]
C28[]
C28[]
C13[]
C13[]
C13N1
C13N1
C29[]
C29[]
C29[]
C30[]
C30[]
C30[]
C25[]
C25[]
C25[]
C22[]
C22[]
C22[]
C16[]
C16[]
C16[]
C29[]
C29[]
C29[]
C29[]
C29[]
C29[]
C3[]
C3[]
C3N1
C3N1
C3N1
C6[]
C6[]
C6N1
C6N1
C10[]
C10[]
C10N3
C10N3
C10N3
C6[]
C6[]
C6N1
C6N1
C6N1
C17[]
C17[]
C17[]
C16[]
C16[]
C16[]
C5[]
C5[]
C5N2
C3[]
C3[]
C3N2
C3N2
C15[]
C15[]
C15[]
C12[]
C12[]
C12[]
C6[]
C6[]
C6[]
C5[]
C5[]
C5[]
C3[]
C3[]
C3[]
C3[]
C6[]
C6[]
C6[]
C11[]
C11[]
C11[]
C6[]
C6[]
C6[]
C1[]
C1[]
C1[]
C1[]
C2[]
C2[]
C2[]
C2[]
C26[]
C26[]
C26N1
C26N1
C26N1
C5[]
C5[]
C5[]
C5[]
C8[]
C8[]
C8[]
C8[]
C8[]
C8N1
C8N1
C5[]
C5[]
C5[]
C3[]
C3[]
C3[]
C3[]
C3[]
C3[]
C3[]
C3[]
C3[]
C4[]
C4[]
C4N2
C4N2
C16[]
C16[]
C16[]
C5[]
C5[]
C5[]
C10[]
C10[]
C10[]
C6[]
C6[]
C6[]
C12[]
C12[]
C12[]
C5[]
C5[]
C5[]
C3[]
C3[]
C3[]
C4[]
C4[]
C4[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C18[]
C18[]
C18[]
C6[]
C6[]
C6N1
C6N1
C16[]
C16[]
C16[]
C3[]
C3[]
C3[]
C9[]
C

C5[]
C5[]
C27[]
C27[]
C27[]
C27[]
C17[]
C17[]
C17N3
C17N3
C17N3
C17[]
C17[]
C17N3
C17N3
C17N3
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C27[]
C27[]
C27[]
C27[]
C27[]
C27[]
C27[]
C27[]
C6[]
C6[]
C6[]
C35[]
C35[]
C35N7
C35N7
C35N7
C35N7
C35[]
C35[]
C35N7
C35N7
C35N7
C35N7
C35[]
C35[]
C35N7
C35N7
C35N7
C35N7
C1[]
C1[]
C1[]
C1[]
C6[]
C6[]
C6[]
C9[]
C9[]
C9N1
C9N1
C11[]
C11[]
C11N2
C11N2
C47[]
C47[]
C47[]
C47[]
C47[]
C47[]
C42[]
C42[]
C42[]
C47[]
C47[]
C47[]
C41[]
C41[]
C41[]
C41[]
C41[]
C41[]
C41[]
C41[]
C41[]
C36[]
C36[]
C36[]
C30[]
C30[]
C30[]
C36[]
C36[]
C36[]
C30[]
C30[]
C30[]
C36[]
C36[]
C36[]
C42[]
C42[]
C42[]
C42[]
C42[]
C42[]
C47[]
C47[]
C47[]
C48[]
C48[]
C48[]
C42[]
C42[]
C42[]
C35[]
C35[]
C35N7
C35N7
C35N7
C35N7
C63[]
C63[]
C63N1
C63N1
C63N1
C6[]
C6[]
C6[]
C5[]
C5[]
C5[]
C6[]
C6[]
C6[]
C6[]
C6[]
C6[]
C10[]
C10[]
C10[]
C11[]
C11[]
C11[]
C11[]
C11[]
C11[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C5[]
C39[]
C39[]
C39[]
C39[]
C35[]
C35[]
C35N

...

RXN-11315-XYLITOL/NAD//CPD-15377/NADH/PROTON.35.
NAD_c + XYLITOL_c <=> CPD__45__15377_c + NADH_c + PROTON_c
D-XYLULOSE-REDUCTASE-RXN
NAD_c + XYLITOL_c <=> D__45__XYLULOSE_c + NADH_c + PROTON_c
RXN-11825
NAD_c + XYLITOL_c <=> L__45__XYLULOSE_c + NADH_c + PROTON_c
RXN-8773-XYLITOL/NADP//BETA-D-XYLOSE/NADPH/PROTON.41.
NADP_c + XYLITOL_c <=> BETA__45__D__45__XYLOSE_c + NADPH_c + PROTON_c
RXN-11315-XYLITOL/NADP//CPD-15377/NADPH/PROTON.37.
NADP_c + XYLITOL_c <=> CPD__45__15377_c + NADPH_c + PROTON_c
RXN-8773
NADP_c + XYLITOL_c <=> D__45__Xylopyranose_c + NADPH_c + PROTON_c
L-XYLULOSE-REDUCTASE-RXN
NADP_c + XYLITOL_c <=> L__45__XYLULOSE_c + NADPH_c + PROTON_c
RXN-8773-XYLITOL/NADP//XYLOSE/NADPH/PROTON.34.
NADP_c + XYLITOL_c <=> NADPH_c + PROTON_c + XYLOSE_c
1.1.3.41-RXN
OXYGEN__45__MOLECULE_c + XYLITOL_c --> HYDROGEN__45__PEROXIDE_c + XYLOSE_c
XYLISOM-RXN-XYLOSE//D-XYLULOSE.19.
XYLOSE_c <=> D__45__XYLULOSE_c
D__45__XYLOSE__45__1__45__DEHYDROGENASE__45__RXN__45__XYLOSE__47__NAD__47____47__CPD_

RXN__45__18025__45__CPD__45__18963__47__CPD__45__9968__47__WATER__47____47__CPD__45__18965__47__CPD__45__12119__47__PROTON__46__53__46__
CPD__45__18963_c + 2.0 CPD__45__9968_c + WATER_c --> 2.0 CPD__45__12119_c + CPD__45__18965_c + PROTON_c
RXN__45__18025__45__CPD__45__18963__47__CPD__45__9969__47__WATER__47____47__CPD__45__18965__47__CPD__45__12120__47__PROTON__46__53__46__
CPD__45__18963_c + 2.0 CPD__45__9969_c + WATER_c --> 2.0 CPD__45__12120_c + CPD__45__18965_c + PROTON_c
RXN__45__18025__45__CPD__45__18963__47__CPD__45__9970__47__WATER__47____47__CPD__45__18965__47__CPD__45__12121__47__PROTON__46__53__46__
CPD__45__18963_c + 2.0 CPD__45__9970_c + WATER_c --> 2.0 CPD__45__12121_c + CPD__45__18965_c + PROTON_c
RXN__45__18025__45__CPD__45__18963__47__CPD__45__9971__47__WATER__47____47__CPD__45__18965__47__CPD__45__12122__47__PROTON__46__53__46__
CPD__45__18963_c + 2.0 CPD__45__9971_c + WATER_c --> 2.0 CPD__45__12122_c + CPD__45__18965_c + PROTON_c
RXN__45__18025__45__CPD__45__18963__

N__45__ACETYLGLUCOSAMINE__45__KINASE__45__RXN__45__N__45__acetyl__45__D__45__glucosamine__47__ATP__47____47__CPD__45__16167__47__ADP__47__PROTON__46__49__46__
ATP_c + N__45__acetyl__45__D__45__glucosamine_c --> ADP_c + CPD__45__16167_c + PROTON_c
N__45__ACETYLGLUCOSAMINE__45__KINASE__45__RXN__45__N__45__acetyl__45__D__45__glucosamine__47__ATP__47____47__CPD__45__16168__47__ADP__47__PROTON__46__49__46__
ATP_c + N__45__acetyl__45__D__45__glucosamine_c --> ADP_c + CPD__45__16168_c + PROTON_c
RXN__45__18662__45__N__45__ACETYL__45__D__45__GLUCOSAMINE__47__ATP__47____47__N__45__ACETYL__45__D__45__GLUCOSAMINE__45__1__45__P__47__ADP__47__PROTON__46__66__46__
ATP_c + N__45__ACETYL__45__D__45__GLUCOSAMINE_c --> ADP_c + N__45__ACETYL__45__D__45__GLUCOSAMINE__45__1__45__P_c + PROTON_c
RXN0__45__0__45__N__45__ACETYLNEURAMINATE__47____47__N__45__ACETYLNEURAMINATE__46__41__46__
N__45__ACETYLNEURAMINATE_e --> N__45__ACETYLNEURAMINATE_p
TRANS__45__RXN__45__25__45__N__45__ACETYLNEURAMINATE__47__PROTON__

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



finish!!
